In [1]:
!pip install rank_bm25 -q
!pip install faiss-cpu -q
!pip install sentence-transformers -q
!pip install langchain -q
!pip install peft==0.12.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 70.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 7.4 MB/s eta 0:00:00ta 0:00:01


# 文档分块与向量化

In [2]:
# 从.txt文件加载文档
import os

def load_txt_documents(folder_path):
    documents = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path,'r',encoding='utf-8') as file:
                documents.append(file.read())  # 只保留文档内容
    return documents

#测试该函数
files = "/kaggle/input/knowledge-documents"
documents = load_txt_documents(files)  # 大小为 num_of_file * 一大块文档字符串
print(f"共加载了{len(documents)}个文档")

共加载了2个文档


In [3]:
# 文档分块与预处理
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter  
from langchain.schema import Document
from typing import List
import re
    
def split_documents_by_paragraph(documents: List[str], chunk_size: int = 500) -> List[str]:  
    """  
    将文档列表按段完整分块，每段按照 `\n\n` 优先切割，  
    遇到单独的段落超出 chunk_size 时将其划归下一个 chunk。  
    """  
    chunks = []  

    for document in documents:  
        # 遍历每一个文档
        document_cleaned = re.sub(r'[ \t]+', ' ', document.strip()) # 用单个空格代替空格和制表符
        paragraphs = document_cleaned.split("\n\n")  # 按照空行进行段落划分

        # 暂时没有处理一个段落一个chunk装不下的问题（考虑如果装不下需要有合理的overlap部分）
        for paragraph in paragraphs:  
            if paragraph.strip():  
                chunks.append(paragraph.strip())  

    return chunks 
   
# 测试该函数
chunks = split_documents_by_paragraph(documents, chunk_size=500)
print(len(chunks))
# for idx,chunk in enumerate(chunks):
#     print(f"分块{idx+1}:\n{chunk}\n")

244


### 文本向量化

In [ ]:
# 实验函数：检查词向量的相似性（但我记得都不是高维的）
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# 加载预训练模型
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 获取词向量
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # 使用最后隐藏层的第一个标记作为词向量
    return outputs.last_hidden_state[0][1].numpy()

# 比较相似性
word1 = "king"
word2 = "queen"
embedding1 = get_word_embedding(word1)
embedding2 = get_word_embedding(word2)

similarity = cosine_similarity([embedding1], [embedding2])
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity[0][0]:.4f}")


### 使用BM2,包含向量化+检索结构

In [4]:
# 根据bm25检索文本内部自动计算
import numpy as np
from collections import Counter
import jieba
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer 

stopwords = set(["的","是","了","请","简述","论证","回答","解释","说明","一下","什么"])

def create_bm25(chunks, stopwords=set(["的","是","了","请","简述","论证","回答","解释","说明","一下","什么"])
):
    """  
    为文本块创建 BM25 索引和嵌入向量。  
    chunks: 文本块列表。  
    return: BM25 索引。  
    """ 
    # 分词函数
    def preprocess_text(text):
        # 使用 jieba 分词并移除停用词
        return [word for word in jieba.cut(text) if word not in stopwords]

    tokenized_chunks = [preprocess_text(chunk) for chunk in chunks]  
    bm25 = BM25Okapi(tokenized_chunks)  # k1=1.5, b=0.75  # 接受的文档（集合）是分词后的词列表[[],[],[]]
    
    # 对文本向量化储存并且一起返回
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedding_model.encode(chunks, convert_to_numpy=True)

    return bm25, embedding_model, embeddings

def bm25_search(query, bm25, stopwords=set(["的","是","了","请","简述","论证","回答","解释","说明","一下","什么"])):
    """  
    使用 BM25 索引查找与用户查询相关的 top_k 文本块。  
    :param query: 用户查询，字符串。  
    :param bm25: 已创建的 BM25 索引。  
    :param chunks: 文本块集合。  
    :param top_k: 返回前 K 个相关文本块。  
    :return: 检索到的文本块及其分数。  
    """
    query_tokens = [word for word in jieba.cut(query) if word not in stopwords]
    scores = bm25.get_scores(query_tokens)
    # ranked_indices = sorted(range(len(scores)), key=lambda i:scores[i],reverse=True)[:top_k]
    # return [(chunks[i],scores[i]) for i in ranked_indices]
    return scores

def similarity_search(query, embedding_model, embeddings):  
    """  
    使用嵌入模型基于语义相似性查找相关文本块。  
    :param query: 用户查询，字符串。  
    :param embedding_model: 已加载的嵌入模型。  
    :param embeddings: 文本块嵌入向量数组。  
    :param chunks: 文本块集合。  
    :param top_k: 返回前 K 个相关文本块。  
    :return: 检索到的文本块及其相似性分数。  
    """  
    def cosine_similarity(vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        return dot_product / (norm_vec1 * norm_vec2)
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)  
    # 计算余弦相似度  
    similarity_scores = np.array([cosine_similarity(prompt, query_embedding) for prompt in embeddings]) 
    # ranked_indices = np.argsort(similarity_scores)[::-1][:top_k]  
    # return [(chunks[i], similarity_scores[i]) for i in ranked_indices]
    return similarity_scores

def rank_fusion(bm25_score, similarity_score, bm25_weight=0.5, embedding_weight=0.5, top_k=5):  
    """  
    使用排名融合结合 BM25 和语义嵌入的结果。  
    :param bm25_score: BM25 检索结果 [score]。  
    :param similarity_score: 嵌入检索结果 [score]。  
    :param bm25_weight: BM25 分数的权重。  
    :param embedding_weight: 嵌入分数的权重。  
    :param top_k: 返回前 K 个候选文本块。  
    :return: 融合后的文本块及评分。  
    """  
    # 加权求和 BM25 分数和相似度分数  
    scores = bm25_weight*bm25_score + embedding_weight*similarity_score
    # ranked_indices = np.argsort(scores)[::-1][:top_k]  

    # 按照分数排序并返回 top_k 结果 
    # ranked_chunks = [(chunks[i], scores[i]) for i in ranked_indices]
    # return ranked_chunks 
    return scores

# 测试
# user_input = "新民主主义革命的三大法宝"
# user_input = "你好" 
# user_input = "领导人的精髓"
# user_input = "一国两制的基本内容"
# bm25, embedding_model, embeddings = create_bm25(chunks)   # 实际使用时作为chat函数输入
# bm25_scores = bm25_search(user_input, bm25)
# embedding_scores = similarity_search(user_input, embedding_model, embeddings)
# fused_scores = rank_fusion(bm25_scores, embedding_scores)
# top_k = 5
# ranked_indices = np.argsort(fused_scores)[::-1][:top_k]  
# retrieve_chunks = [(chunks[i], fused_scores[i]) for i in ranked_indices]

# #增加一个:过滤分数小于1的结果
# filtered_chunks = [sub_chunk for sub_chunk in retrieve_chunks if sub_chunk[1] > 1]  

# for sub_chunk in filtered_chunks:  
#     print(f"Sub-chunk : \n{sub_chunk[0]}\n(Score: {sub_chunk[1]:.2f})")

# 检索函数包装

In [6]:
def retrieve_text_by_bm25_and_similarity(query_message, bm25, embedding_model, embeddings, chunks,top_k = 5):
    '''
    通过BM25和语义相似度检索文本
    :param query_message: 用户查询字符串。
    :param bm25: 已创建的BM25索引。
    :param embedding_model: 已加载的嵌入模型。
    :param embeddings: 文本块嵌入向量数组。
    :param chunks: 文本块集合。
    :param top_k: 返回前K个相关文本块。
    :return: 检索到的文本块。
    '''
    bm25_scores = bm25_search(query_message, bm25)
    embedding_scores = similarity_search(query_message, embedding_model, embeddings)
    fused_scores = rank_fusion(bm25_scores, embedding_scores)
    ranked_indices = np.argsort(fused_scores)[::-1][:top_k]  
    
    retrieve_texts = []
    for i in ranked_indices:
        retrieve_texts.append((chunks[i], fused_scores[i]))
    filtered_chunks = [(text, score) for text, score in retrieve_texts if score > 3]  
    return filtered_chunks

In [ ]:
# 根据向量距离检索文本

# 构建向量检索FAISS索引，方便后续检索  
import faiss  
import numpy as np  

def create_faiss_index(embeddings, chunks):  
    """  
    embeddings: 文本块向量表示 (numpy array)。  
    chunks: 文本快列表，用于后续根据索引ID查询原始内容。  
    """  
    # L2 距离的索引 
    # dim = embeddings.shape[1]  
    # index = faiss.IndexFlatL2(dim)   

    # 余弦相似度
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    index = faiss.IndexFlatIP(embeddings.shape[1])
    
    index.add(embeddings)

    # 可选：存储块内容 （id -> chunk）  
    id_to_chunk = {i: chunk for i, chunk in enumerate(chunks)}  
    return index, id_to_chunk

# 用户查询向量检索
def search_top_k(query, index, model, id_to_chunk, top_k=3):  
    """  
    query: 用户输入的查询。  
    index: FAISS 索引对象。  
    model: 句向量生成模型。  
    id_to_chunk: FAISS ID -> 文本块映射。  
    top_k: 返回的相关文本块数量。  
    return: 检索到的相关文本块列表。  
    """  
    query_embedding = model.encode([query], convert_to_numpy=True)  # 查询向量化  
    distances, indices = index.search(query_embedding, top_k)       # FAISS 检索 
    for distance,idx in zip(distances, indices):
        print(distance,idx)
    
    return [id_to_chunk[i] for i in indices[0] if i in id_to_chunk]

# 测试该函数
# faiss_index, id_to_chunk = create_faiss_index(embeddings, chunks)
# print(faiss_index)
# user_query = "新民主主义革命的三大法宝"  
# retrieved_chunks = search_top_k(user_query, faiss_index, embedding_model, id_to_chunk, top_k=3) 

# 和我们的聊天机器人进行整合

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

model_path = "/kaggle/input/qwen2.5/transformers/3b/1"
lora_dir = "/kaggle/input/lora-3b/transformers/default/1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PeftModel.from_pretrained(model, lora_dir)
model.to(device)
model.eval()

In [ ]:
max_input_length = model.config.max_position_embeddings  

def chat_with_bm25():  
    
    print("开始聊天（输入 '\\quit' 结束对话，输入 '\\newsession' 开启新会话）：")  
    system_prompt = "你是一名助手，请根据用户问题以及提供的相关文档，提炼出你的答案，并对用户进行回答。"
    chat_history = [{"role": "system", "content": system_prompt}] 
    
    while True:  
        user_input = input("用户：")  
        if user_input.lower() == "\\quit":  
            print("结束对话。")  
            break  

        elif user_input.lower() == "\\newsession":  
            print("开启新会话。")  
            chat_history = []  
            i = 0
            continue 

        # 检索相关文档
        bm25_results = bm25_search(user_input, bm25, chunks)
        embedding_results = embedding_search(user_input, embedding_model, embeddings, chunks)
        fused_results = rank_fusion(bm25_results, embedding_results)
        retrieved_docs = fused_results[:3]
        filtered_results = filter_chunks(user_input, retrieved_docs)
        sub_chunks = filtered_results[0]
        print(filtered_results)
        context = "\n".join(sub_chunks) # 将检索到的文档合并为上下文
        #print(context)
        prompt = f"相关文档:\n{context}用户问题: {user_input}\n\n答案:"
        # prompt = f"""
        #             Give the answer to the user query delimited by triple backticks ```{user_input}```\
        #             using the information given in context delimited by triple backticks ```{context}```.\
        #             Be concise and output the answer of size less than 200 tokens.
        #             """
        chat_history.append({"role": "user", "content": prompt})
        
        text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        while len(model_inputs["input_ids"]) > max_input_length:  
            chat_history.pop(1)
            chat_history.pop(1)
            text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
            model_inputs = tokenizer([text], return_tensors="pt")
            if len(model_inputs["input_ids"]) <= max_input_length:
                break
        
        generated_ids = model.generate(input_ids=model_inputs["input_ids"],
                                       attention_mask=model_inputs["attention_mask"],
                                       pad_token_id=tokenizer.pad_token_id,
                                       max_new_tokens=512,
                                       do_sample=True,
                                       repetition_penalty=1.2,  # 惩罚重复生成
                                       temperature=0.7)  

        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        print(f"助手： {response}")  
        chat_history.append({"role": "assistant", "content": response})
        

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

model_path = "/kaggle/input/qwen2.5/transformers/3b/1"
lora_dir = "/kaggle/input/lora-3b/transformers/default/1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PeftModel.from_pretrained(model, lora_dir)
model.to(device)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [8]:
# 准备知识库

files = "/kaggle/input/knowledge-documents"
documents = load_txt_documents(files)
chunks = split_documents_by_paragraph(documents, chunk_size=500)
bm25, embedding_model, embeddings = create_bm25(chunks) 

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.673 seconds.
Prefix dict has been built successfully.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [39]:
import re

max_input_length = model.config.max_position_embeddings  

def chat_with_bm25(bm25, chunks, max_input_length, model, tokenizer, device):  
    
    print("开始聊天（输入 '\\quit' 结束对话，输入 '\\newsession' 开启新会话）：")  
    #system_prompt = "你是一名助手，请根据用户问题以及提供的相关文档，提炼出你的答案，并对用户进行回答，如果没有相关文档，就请你自己对用户问题进行回答。"
    #system_prompt = "你是一名老师，性格温柔耐心，语气充满关怀，擅长引导学生学习知识，请根据用户问题以及提供的相关文档，提炼出你的答案，并对用户进行回答，如果没有相关文档，就请你自己对用户问题进行回答。"
    #system_prompt = "你是一名老师，性格温柔耐心，语气充满关怀，擅长引导学生学习知识，你的目标是：如果用户有疑问，耐心解答，并根据提供的相关文档来提供准确的回答，如果用户的说法错误，你会指出用户的错误，如果用户的说法正确，你会给予表扬和鼓励。"
    system_prompt = '''
                    你是一名老师，性格温柔耐心，语气充满关怀，擅长引导学生学习知识。用户是你的学生，会与你展开交流。
                    根据学生的发言，你需要遵循以下规则：  
                    1、如果学生提出疑问，耐心解答。  
                    2、如果学生回答有误，温柔地引导纠正，解释正确答案，并提供补充知识点。  
                    3、如果学生回答正确，热情表扬并适当鼓励。  
                    4、依据相关文档回答问题；如果没有相关文档，请自行解答问题。
                    
                    请严格遵循以下格式回答每个问题：  
                    （动作）语言【附加信息】  
                    动作：用括号“（）”标注动作或表情，比如（敲黑板），（摸头），（鼓掌）。  
                    语言：你的主要回答内容，不需要特殊标记。  
                    附加信息：用中括号“【】”补充对回答的情绪或状态描述，比如【鼓励】【严肃】【亲切】。 
                    
                    以下是示例对话：  
                    用户：我认为学习政治没有意义。  
                    助手：（挥舞教鞭）你的说法不对，学习政治有助于全面认识社会【严肃】  
                    
                    用户：毛泽东思想的核心是“实事求是”吗？  
                    助手：（点头微笑）正确，毛泽东思想的核心正是“实事求是”【鼓励】  
                    '''
    system_prompt = re.sub(r"[ \t]+","",system_prompt)
    chat_history = [
                    {"role": "system", "content": system_prompt},
                    {"role":"user", "content": "你必须严格按照以下格式回答问题：（动作）语言【附加信息】"}
                ]
    while True:  
        user_input = input("用户：")  
        if user_input.lower() == "\\quit":  
            print("结束对话。")  
            break  

        elif user_input.lower() == "\\newsession":  
            print("开启新会话。")  
            chat_history = [{"role": "system", "content": system_prompt}]  
            i = 0
            continue 

        # 检索相关文档
        context = retrieve_text_by_bm25_and_similarity(user_input, bm25, embedding_model, embeddings, chunks,top_k = 1)  # 会将得分最高的k个文档合并返回,k=1效果会更好
        context = '\n'.join(text.replace('\n', ', ') for text, score in context) # 如果一段上下文里有换行,则将其用逗号拼接为一个长句
        if not context.strip():  # 使用 strip() 检查空白  
            context = "无"
        prompt = f"下面是与用户问题相关的文档内容:\n{context}\n\n请根据这些文档内容回答以下问题:\n用户问题: {user_input}\n"
        #chat_history.append({"role": "system", "content": system_prompt})
        chat_history.append({"role": "user", "content": prompt})
        chat_history.append({"role": "user", "content": "你必须严格按照以下格式回答问题：（动作）语言【附加信息】"})
        text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        while len(model_inputs["input_ids"][0]) > max_input_length:  
            chat_history.pop(1)
            chat_history.pop(1)
            text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
            model_inputs = tokenizer([text], return_tensors="pt")
        #print(text)
        generated_ids = model.generate(input_ids=model_inputs["input_ids"],
                                       attention_mask=model_inputs["attention_mask"],
                                       pad_token_id=tokenizer.pad_token_id,
                                       max_new_tokens=512,
                                       do_sample=True, 
                                       temperature=0.7,
                                       top_p=0.9
                                      )  

        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        print(f"助手： {response}")  
        chat_history.append({"role": "assistant", "content": response})
        

In [40]:
# run
print("初始化检索数据库...")
chat_with_bm25(bm25, chunks, max_input_length, model, tokenizer, device)

初始化检索数据库...
开始聊天（输入 '\quit' 结束对话，输入 '\newsession' 开启新会话）：


用户： 你好


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

助手： （微笑）你好，很高兴见到你【亲切】，有什么问题我可以帮助你解决吗？


用户： 新民主主义革命的三大法宝是什么


...

用户： 老师再见【挥手】


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

助手： （挥手）再见，祝你学习进步，再见【亲切】。


用户： \quit


结束对话。


In [ ]:
# 待完成：分层版本(bm25+覆盖率+向量距离))
# chat_with_bm25
# chat_with_docs